In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import datetime
import time
import predict
import base

In [ ]:
#读取user_pay_count
user_pay_counts = base.ReadUserPayCountCSV();
#读取开店时间数据
shop_open_dates = base.ReadShopOpenDateCSV();
###读取商家信息
shop_infos = base.ReadShopInfoCSV();
##读取城市名称对照表
city_names = base.ReadCityNameCSV();
#读取休假表
calenders = base.ReadCalendarCSV();

In [ ]:
#ARIMA

In [ ]:
predict.initTrainSingleArimaData(88, shop_open_dates, user_pay_counts, pd.to_datetime('2015-7-1'), pd.to_datetime('2016-10-17'))

In [ ]:
predict.initTrainArimasData('validation', shop_open_dates, user_pay_counts, pd.to_datetime('2016-7-1'), pd.to_datetime('2016-10-17'))

In [ ]:
#执行ArimaTrain.R

In [ ]:
arima_df = predict.predictArimas('validation')

In [ ]:
#ARIMA 时间序列预测
evaluations = {}


for id in range(1,2001):
    y_arima = arima_df.ix[id].values
    y_Test = base.getTestCount(user_pay_counts, id, pd.to_datetime('2016-10-18'), pd.to_datetime('2016-10-31')).values
    e = base.evaluation_tm(y_Test, y_arima)
    print("id=%d evaluation=%f"%(id, e))
    evaluations[id] = e
    
arima_evaluation_df = pd.DataFrame.from_dict(evaluations, orient='index');
arima_evaluation_df.columns = ['evaluation']
arima_evaluation_df.index.name = 'id'
arima_evaluation_df.sort_values('evaluation')
arima_evaluation_df.to_csv("./data/dataset/dataset/validation/arima_evaluation.txt", sep='\t', index=True, encoding='UTF-8')

In [ ]:
arima_evaluation_df['evaluation'].mean()

In [ ]:
arima_evaluation_df[arima_evaluation_df['evaluation'].isnull()]

In [ ]:
#Holiday-Mean

In [ ]:
holiday_mean_df = predict.trainHolidayMeansData(
    shop_open_dates, user_pay_counts, calenders, pd.to_datetime('2016-9-27'), pd.to_datetime('2016-10-17'))

In [ ]:
start_time = pd.to_datetime('10/18/2016');
end_time = pd.to_datetime('10/31/2016');
predict_date = pd.date_range(start=start_time, end=end_time, freq='D', normalize=True)
pridicts = {}
evaluations = {}
for id in range(1, 2001):
    y_Pred = []
    for date in predict_date:
        daytype = calenders.loc[date.strftime('%Y-%m-%d')]['daytype']
        y_Pred.append(holiday_mean_df.loc[id][daytype]);
    pridicts[id] = y_Pred
    y_Test = base.getTestCount(user_pay_counts, id, start_time, end_time).values
    e = base.evaluation(y_Test, y_Pred)
    print("id=%d evaluation=%f"%(id, e))
    pridicts[id] = y_Pred
    evaluations[id] = e

In [ ]:
evaluation_df = pd.DataFrame.from_dict(evaluations, orient='index');
evaluation_df.columns = ['evaluation']
evaluation_df.index.name = 'id'
evaluation_df.sort_values('evaluation')

In [ ]:
evaluation_df = evaluation_df.fillna(1)
evaluation_df.to_csv("./data/dataset/dataset/validation/holiday_mean_evaluation.txt", sep='\t', index=True, encoding='UTF-8')

In [ ]:
evaluation_df['evaluation'].mean()